In [1]:
import numpy as np 
import pandas as pd
from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Class of Neural Network with Affine Layer 

In [29]:
class NeuralNetwork:
    def __init__(self):
        self.temp=0
        self.weights=[]
        self.bias=[]
        self.outputs=0
        self.derivative_table=0
        self.model_loss=100
        self.local_derivatives=[]
    def initialize_weights_and_bais(self,X,layers,neurons):
        l=X.shape[1]
        for i,j in enumerate(neurons):
            self.weights.append(np.array([np.random.random(l) for k in range(j)]))
            self.bias.append(np.round(np.random.random(j),6))
            l=self.weights[i].shape[0]
    def ReLU(self,INPUT):
        INPUT[INPUT<0]=0
        derr=np.zeros_like(INPUT)
        derr[INPUT>0]=1
        return INPUT,derr
    def forward(self,X):
        a=X
        for i in range(len(self.weights)):
            dw=a
            a=np.dot(a,self.weights[i].T)+self.bias[i]
            dx=self.weights[i]
            db=np.ones(self.bias[i].shape)
            if i!=len(self.weights)-1:
                relu,relu_derr=self.ReLU(a)
            else :
                relu,relu_derr=np.array([0]),np.array([0])
            self.local_derivatives.append([relu,dw,dx,db,relu_derr])
#             if i!=len(self.weights)-1:
#                 a=relu
        return a
    
    def loss(self,y,outputs):
        self.derivative_table=outputs-outputs[np.arange(y.shape[0]),y].reshape(-1,1)
        self.derivative_table[self.derivative_table<=0]=0
        self.model_loss=np.sum(self.derivative_table)
        self.derivative_table[self.derivative_table>0]=1
        self.derivative_table[np.arange(y.shape[0]),y]=np.sum(self.derivative_table,axis=1)*-1
        return self.derivative_table
        
    def backward(self,LR,output):
        holding_weights=[]
        holding_bias=[]
        final_output=output
        c=c=len(self.weights)-1
        for i in reversed(self.local_derivatives):
            if i[-1].shape[0]!=1:
                final_ouput=np.multiply(final_output,i[-1])       
            self.weights[c]=self.weights[c]-(LR*np.dot(final_output.T,i[1]))
            self.bias[c]=self.bias[c]-(LR*np.sum(final_output,axis=0))
            final_output=np.dot(final_output,i[2])
            c=c-1
            
    def train(self,X,y,layers,neurons,LR=0.00001,iterations=50000,loss_time=1000):

        self.initialize_weights_and_bais(X,layers,neurons)
        for i in range(iterations):
            OUT=self.forward(X)
            DERR=self.loss(y,OUT)
            self.backward(LR,DERR)

            self.local_derivatives=[]            # empty local derivatives to calculate them again
            if i%loss_time==0:
                print('ITERATION : ',i,' MODEL LOSS : ',self.model_loss)
    
    def test(self,X,y):
        temp=self.forward(X)
        holder=[]
        for i in temp:
            i=list(i)
            holder.append(i.index(max(i)))
        tt = sum(1 for x, z in zip(holder, y) if x == z)
        print((tt/len(holder))*100)

In [30]:
data=pd.read_csv('Housing.csv')


In [31]:
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


# Preparing Dataset

In [32]:
# data=datasets.load_iris()
x=np.array(data[['area','bedrooms']][:2000])
y=np.array(data['price'][:2000])
# x,y,names=data['data'],data['target'],data["target_names"]

In [33]:
x,y

(array([[7420,    4],
        [8960,    4],
        [9960,    3],
        ...,
        [3620,    2],
        [2910,    3],
        [3850,    3]]),
 array([13300000, 12250000, 12250000, 12215000, 11410000, 10850000,
        10150000, 10150000,  9870000,  9800000,  9800000,  9681000,
         9310000,  9240000,  9240000,  9100000,  9100000,  8960000,
         8890000,  8855000,  8750000,  8680000,  8645000,  8645000,
         8575000,  8540000,  8463000,  8400000,  8400000,  8400000,
         8400000,  8400000,  8295000,  8190000,  8120000,  8080940,
         8043000,  7980000,  7962500,  7910000,  7875000,  7840000,
         7700000,  7700000,  7560000,  7560000,  7525000,  7490000,
         7455000,  7420000,  7420000,  7420000,  7350000,  7350000,
         7350000,  7350000,  7343000,  7245000,  7210000,  7210000,
         7140000,  7070000,  7070000,  7035000,  7000000,  6930000,
         6930000,  6895000,  6860000,  6790000,  6790000,  6755000,
         6720000,  6685000,  6650000,

# Splitting the Dataset into Training and Testing

In [34]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle=True)

In [35]:
type(x_train)

numpy.ndarray

# Training the Model 

In [36]:
a=NeuralNetwork()
a.train(x_train,y_train,3,[5,7,1],0.0001,50000,1000)

IndexError: index 3745000 is out of bounds for axis 1 with size 1

# Testing the Model

In [6]:
a.test(x_test,y_test)

93.33333333333333


In [7]:
import pickle
with open('model.pkl', 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(a, outp, pickle.HIGHEST_PROTOCOL)